## Data Functions

In [ ]:
kinship_file = "/locus/home/ddynerman/fine-mapping/data/ukbb/ukb_rel_a65528_s488171.dat"

In [ ]:
library(readr)
library(dplyr)
library(tidyr)
library(purrr)
library(ggplot2)
library(data.table)

In [ ]:
make_kin_df <- function(kinship_file){
    
    dt_king <- data.table::fread(kinship_file, sep = " ")
    dt_king <- dt_king %>%
        mutate(pairs = paste(ID1, ID2, " ")) %>%
        mutate(relationship = case_when(Kinship < 1/(2^(9/2)) ~ "Unrelated",
                                between(Kinship, 1/(2^(9/2)), 1/(2^(7/2))) ~ "3rd Degree",
                                between(Kinship, 1/(2^(7/2)), 1/(2^(5/2))) ~ "2nd Degree",
                                between(Kinship, 1/(2^(5/2)), 1/(2^(3/2))) ~ "1st Degree",
                                 Kinship > 1/(2^(3/2)) ~ "Monozygotic twin"))
    return(dt_king)
}

In [ ]:
subset_related <- function(triangle_file, kin_df){
    
    tri_dt <- data.table::fread(triangle_file)
    rel_pairs <- tri_dt %>% 
        mutate(pairs1 = paste(ID1, ID2, " ")) %>% 
        mutate(pairs2 = paste(ID2, ID1, " "))

    matching_pairs1 <- rel_pairs %>%
        filter(pairs1 %in% kin_df$pairs)
    head(matching_pairs1)

    matching_pairs2 <- rel_pairs %>%
        filter(pairs2 %in% kin_df$pairs)
    head(matching_pairs2)

    pairs1 <- select(matching_pairs1, pairs1, rel) %>% 
        rename(pairs = pairs1)
    pairs2 <- select(matching_pairs2, pairs2, rel) %>% 
        rename(pairs = pairs2)
    
    related_df <- rbind(pairs1, pairs2) %>%
        left_join(kin_df)
    
    return(related_df)
    
}



In [ ]:
## rewrite:
# iterate over all pairs you want to keep first
# look up in big matrix by [i,j]
# i think this will actually make more sense in python
subset_related <- function(rel_file, kin_df){
    
    tri_dt <- data.table::fread(triangle_file)
    rel_pairs <- tri_dt %>% 
        mutate(pairs1 = paste(ID1, ID2, " ")) %>% 
        mutate(pairs2 = paste(ID2, ID1, " "))

    matching_pairs1 <- rel_pairs %>%
        filter(pairs1 %in% kin_df$pairs)
    head(matching_pairs1)

    matching_pairs2 <- rel_pairs %>%
        filter(pairs2 %in% kin_df$pairs)
    head(matching_pairs2)

    pairs1 <- select(matching_pairs1, pairs1, rel) %>% 
        rename(pairs = pairs1)
    pairs2 <- select(matching_pairs2, pairs2, rel) %>% 
        rename(pairs = pairs2)
    
    related_df <- rbind(pairs1, pairs2) %>%
        left_join(kin_df)
    
    return(related_df)
    
}

In [ ]:
subset_unrelated <- function(triangle_file, kin_df){
    
    `%!in%` <- Negate(`%in%`)
    
    tri_dt <- data.table::fread(triangle_file)
    rel_pairs <- tri_dt %>% 
        mutate(pairs1 = paste(ID1, ID2, " ")) %>% 
        mutate(pairs2 = paste(ID2, ID1, " "))

    unrelated_pairs <- rel_pairs %>%
        filter(pairs1 %!in% king_pairs$pairs & pairs2 %!in% king_pairs$pairs)
    
    unrelated_df <- unrelated_pairs %>%
    select(ID1, ID2, rel) %>%
    mutate(Kinship = 0) %>%
    mutate(relationship = "Unrelated")
    
    return(unrelated_df)
    
}

In [ ]:
rel_predict <- function(df){
    
    df_out <- df %>%
        mutate(rel_predict = case_when(rel/2 < 1/(2^(9/2)) ~ "Unrelated",
                                       between(rel/2, 1/(2^(9/2)), 1/(2^(7/2))) ~ "3rd Degree",
                                       between(rel/2, 1/(2^(7/2)), 1/(2^(5/2))) ~ "2nd Degree",
                                       between(rel/2, 1/(2^(5/2)), 1/(2^(3/2))) ~ "1st Degree",
                                       rel/2 > 1/(2^(3/2)) ~ "Monozygotic twin"))
    
    return(df_out)
}

In [ ]:
calc_metrics <- function(df, level){
    
    true_positive <- df %>% 
    filter(relationship == level & rel_predict == level) %>%
    tally()

    false_positive <- df %>% 
        filter(relationship != level & rel_predict == level) %>%
        tally()

    true_negative <- df %>% 
        filter(relationship != level & rel_predict != level) %>%
        tally()

    false_negative <- df %>% 
        filter(relationship == level & rel_predict != level) %>%
        tally()

    precision = true_positive/(true_positive + false_positive)
    recall = true_positive/(true_positive + false_negative)
    FDR = 1 - precision
    specificity = true_negative/(true_negative + false_positive)
    
    df_out <- data.frame(metric = c("precision", "recall", "FDR", "specificity"),
                values = c(precision[1,1], recall[1,1], FDR[1,1], specificity[1,1]),
                level = rep(c(level)))
    
    print(paste("---- METRICS FOR: ", level, " ----"))
    print(paste("precision = ", precision))
    print(paste("recall = ", sensitivity))
    print(paste("FDR = ", FDR))
    print(paste("specificity = ", specificity))
    print(paste("# true positives =", true_positive))
    print(paste("# false positives =", false_positive))
    print(paste("# true negatives =", true_negative))
    print(paste("# false negatives =", false_negative))
    print("\n")
    
    return(df_out)
}

## Plotting Functions

In [ ]:
library(ggplot)
theme_set(theme_bw(base_size = 12))
palette_npg <- c("#E64B35", "#4DBBD5", "#00A087", "#3C5488",
                 "#F39B7F", "#8491B4", "#91D1C2", "#DC0000",
                 "#7E6148", "#B09C85")

In [ ]:
plot_corr <- function(df, title, pal){
    
    ggplot(df, aes(x = Kinship, y = rel, 
                       color = relationship)) +
    geom_point(size = 2, alpha = 0.6) +
    scale_color_manual(values = pal,
                      name = "Relationship") +
    labs(y = "Relatedness", title = title)
    
}

In [ ]:
plot_perf <- function(df, title, pal){
    
    ggplot(all_eval_king11k, aes(x = level, y = values, fill = level)) +
    geom_col(position = "dodge") +
    scale_fill_manual(values = pal) +
    facet_wrap(~metric, scales = "free") +
    theme(legend.position = "top",
          legend.title = element_blank(),
          axis.text.x = element_text(angle = 45, vjust = 1, hjust = 1),
          axis.title.y = element_blank(),
          axis.title.x = element_blank()) +
    labs(title = title)
    
}